In [1]:
import pandas as pd
import numpy as np
import json
import time

from itertools import chain 

In [2]:
%%time
with open(f'KDWD/link_annotated_text.jsonl') as f:
    df = pd.DataFrame(json.loads(line) for line in f)
df

CPU times: user 4min 27s, sys: 21.2 s, total: 4min 48s
Wall time: 4min 54s


,page_id,sections
0,12,"[{'name': 'Introduction', 'text': 'Anarchism i..."
1,25,"[{'name': 'Introduction', 'text': 'Autism is a..."
2,39,"[{'name': 'Introduction', 'text': 'Albedo () (..."
3,290,"[{'name': 'Introduction', 'text': 'A or a is t..."
4,303,"[{'name': 'Introduction', 'text': 'Alabama () ..."
...,...,...
5343560,62470350,"[{'name': 'Introduction', 'text': 'Daming Zhu ..."
5343561,62470423,"[{'name': 'Introduction', 'text': 'Tony Oshey ..."
5343562,62470432,"[{'name': 'Introduction', 'text': '(EC-PL20ZZB..."
5343563,62470465,"[{'name': 'Introduction', 'text': 'Major Gener..."


In [3]:
# Get the introduction text for all pages
df['text'] = [i[0]['text'] for i in df['sections']]
df

,page_id,sections,text
0,12,"[{'name': 'Introduction', 'text': 'Anarchism i...",Anarchism is an anti-authoritarian political a...
1,25,"[{'name': 'Introduction', 'text': 'Autism is a...",Autism is a developmental disorder characteriz...
2,39,"[{'name': 'Introduction', 'text': 'Albedo () (...","Albedo () (, meaning 'whiteness') is the measu..."
3,290,"[{'name': 'Introduction', 'text': 'A or a is t...",A or a is the first letter and the first vowel...
4,303,"[{'name': 'Introduction', 'text': 'Alabama () ...",Alabama () is a state in the southeastern regi...
...,...,...,...
5343560,62470350,"[{'name': 'Introduction', 'text': 'Daming Zhu ...",Daming Zhu is an Assistant Dean for Continuing...
5343561,62470423,"[{'name': 'Introduction', 'text': 'Tony Oshey ...","Tony Oshey Dews (born June 6, 1973) is an Amer..."
5343562,62470432,"[{'name': 'Introduction', 'text': '(EC-PL20ZZB...",(EC-PL20ZZBPRUS) is an sleek design digital co...
5343563,62470465,"[{'name': 'Introduction', 'text': 'Major Gener...",Major General Nils-Fredrik Palmstierna (8 Marc...


In [4]:
%%time
# Export the intro text and page id
df.to_csv('intro_text.csv', columns=['page_id', 'text'], header=['text_id', 'text'], index=False)

CPU times: user 58.7 s, sys: 2.31 s, total: 1min
Wall time: 1min 1s


In [5]:
%%time
# get all the entities and their page_ids from the introduction texts

def get_entities_page_text(section, page_id):
    entities = []
    text = section['text']
    for i, j, k in zip(section['link_offsets'], section['link_lengths'], section['target_page_ids']):
        entities.append([text[i:i+j].strip(), k, page_id])
    return entities

intro_entity_df = pd.DataFrame(list(chain.from_iterable([get_entities_page_text(i[0], j) \
                  for i, j in zip(df['sections'], df['page_id'])])), columns=['entity', 'page_id', 'text_id'])
intro_entity_df

CPU times: user 1min 41s, sys: 2.5 s, total: 1min 43s
Wall time: 1min 43s


,entity,page_id,text_id
0,anti-authoritarian,867979,12
1,political,23040,12
2,social philosophy,586276,12
3,hierarchies,13998,12
4,self-managed,40949353,12
...,...,...,...
35840719,Carl Randall,40277554,62473330
35840720,The World Ends With You,6987282,62473330
35840721,2016 Summer Olympics closing ceremony,44593137,62473330
35840722,2020 Summer Olympics,1610886,62473330


In [6]:
page_df = pd.read_csv('KDWD/page.csv')
page_item_dict = page_df.set_index('page_id').item_id.to_dict()
page_item_dict[12] #6199

6199

In [7]:
intro_entity_df['item_id'] = [page_item_dict[i] for i in intro_entity_df.page_id]
intro_entity_df

,entity,page_id,text_id,item_id
0,anti-authoritarian,867979,12,1030234
1,political,23040,12,179805
2,social philosophy,586276,12,180592
3,hierarchies,13998,12,188619
4,self-managed,40949353,12,15981562
...,...,...,...,...
35840719,Carl Randall,40277554,62473330,16215506
35840720,The World Ends With You,6987282,62473330,1416303
35840721,2016 Summer Olympics closing ceremony,44593137,62473330,18741083
35840722,2020 Summer Olympics,1610886,62473330,181278


In [8]:
%%time
intro_entity_df.to_csv('intro_entity.csv', columns=['entity', 'page_id', 'item_id', 'text_id'], index=False)

CPU times: user 1min 32s, sys: 1.33 s, total: 1min 33s
Wall time: 1min 33s


In [9]:
%%time
entity_df = pd.read_csv('intro_entity.csv')
display(entity_df)
entity_df.dropna(inplace=True)
display(entity_df)
entity_df.to_csv('intro_entity.csv', index=False)

,entity,page_id,item_id,text_id
0,anti-authoritarian,867979,1030234,12
1,political,23040,179805,12
2,social philosophy,586276,180592,12
3,hierarchies,13998,188619,12
4,self-managed,40949353,15981562,12
...,...,...,...,...
35840719,Carl Randall,40277554,16215506,62473330
35840720,The World Ends With You,6987282,1416303,62473330
35840721,2016 Summer Olympics closing ceremony,44593137,18741083,62473330
35840722,2020 Summer Olympics,1610886,181278,62473330


,entity,page_id,item_id,text_id
0,anti-authoritarian,867979,1030234,12
1,political,23040,179805,12
2,social philosophy,586276,180592,12
3,hierarchies,13998,188619,12
4,self-managed,40949353,15981562,12
...,...,...,...,...
35840719,Carl Randall,40277554,16215506,62473330
35840720,The World Ends With You,6987282,1416303,62473330
35840721,2016 Summer Olympics closing ceremony,44593137,18741083,62473330
35840722,2020 Summer Olympics,1610886,181278,62473330


CPU times: user 1min 49s, sys: 2.57 s, total: 1min 52s
Wall time: 1min 53s


## Get all entities and their page_ids from the corpus

In [25]:
#generates the list of entities and corresponding page_ids for each text entry

def get_entities(section):
    entities = []
    text = section['text']
    for i, j, k in zip(section['link_offsets'], section['link_lengths'], section['target_page_ids']):
        entities.append([text[i:i+j], k])
    return entities

def get_section_ids(section):
    return list(chain.from_iterable([get_entities(i) for i in section]))

def get_df_ids(df):
    return list(chain.from_iterable([get_section_ids(i) for i in df.sections]))

In [37]:
%%time
st = time.time()

block_size = 500000
entity_list = []

for i in range(0, len(df), block_size): 
    print('Getting IDs', i, time.time()-st)
    entity_list.append(get_df_ids(df.iloc[i:i+block_size]))
    
len(entity_list)

Getting IDs 0 1.7708463668823242
Getting IDs 500000 91.35793566703796
Getting IDs 1000000 102.20686674118042
Getting IDs 1500000 156.740074634552
Getting IDs 2000000 163.76701641082764
Getting IDs 2500000 170.3705747127533
Getting IDs 3000000 176.58061861991882
Getting IDs 3500000 239.79242420196533
Getting IDs 4000000 245.85486817359924
Getting IDs 4500000 251.7631733417511
Getting IDs 5000000 257.4511179924011
CPU times: user 4min 16s, sys: 5.35 s, total: 4min 21s
Wall time: 4min 21s


11

In [38]:
%%time
entity_df = pd.DataFrame(list(chain.from_iterable(entity_list)), columns=['entity', 'page_id'])
entity_df

CPU times: user 51.2 s, sys: 2.61 s, total: 53.8 s
Wall time: 53.5 s


,entity,page_id
0,anti-authoritarian,867979
1,political,23040
2,social philosophy,586276
3,hierarchies,13998
4,self-managed,40949353
...,...,...
121835448,Carl Randall,40277554
121835449,The World Ends With You,6987282
121835450,2016 Summer Olympics closing ceremony,44593137
121835451,2020 Summer Olympics,1610886


In [39]:
entity_df['item_id'] = [page_item_dict[i] for i in entity_df.page_id]
entity_df

,entity,page_id,item_id
0,anti-authoritarian,867979,1030234
1,political,23040,179805
2,social philosophy,586276,180592
3,hierarchies,13998,188619
4,self-managed,40949353,15981562
...,...,...,...
121835448,Carl Randall,40277554,16215506
121835449,The World Ends With You,6987282,1416303
121835450,2016 Summer Olympics closing ceremony,44593137,18741083
121835451,2020 Summer Olympics,1610886,181278


In [40]:
entity_df.to_feather('entity_df.ftr')

## Get how many times each entity appears in the corpus

In [42]:
page_ids, counts = np.unique(entity_df.page_id, return_counts=True)
page_count_dict = dict(zip(page_ids, counts))
page_count_dict[12]

3540

In [43]:
page_df['counts'] = [page_count_dict.get(i, 0) for i in page_df.page_id]
page_df

,page_id,item_id,title,views,counts
0,12,6199,Anarchism,31335,3540
1,25,38404,Autism,49693,2114
2,39,101038,Albedo,14573,2825
3,290,9659,A,25859,175
4,303,173,Alabama,52765,11125
...,...,...,...,...,...
5362169,62470350,76894635,Daming Zhu,16,0
5362170,62470423,76894633,Tony Dews,7,2
5362171,62470432,76896959,Samsung PL20,9,0
5362172,62470465,6034153,Nils-Fredrik Palmstierna,8,3


In [44]:
page_df.to_csv("id_counts.csv", index=False)